## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [0]:
import tensorflow as tf
tf.enable_eager_execution()

In [0]:
%tensorflow_version 2.x

TensorFlow is already loaded. Please restart the runtime to change versions.


In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


### Collect Data

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd

In [0]:
data = pd.read_csv("/content/drive/My Drive/greatlearning/prices.csv")



### Check all columns in the dataset

In [0]:
data.columns

Index([u'date', u'symbol', u'open', u'close', u'low', u'high', u'volume'], dtype='object')

In [0]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 7 columns):
date      851264 non-null object
symbol    851264 non-null object
open      851264 non-null float64
close     851264 non-null float64
low       851264 non-null float64
high      851264 non-null float64
volume    851264 non-null float64
dtypes: float64(5), object(2)
memory usage: 45.5+ MB


### Drop columns `date` and  `symbol`

In [0]:
data.drop(['date','symbol'],axis =1,inplace = True)

In [0]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [0]:
data_df = data.iloc[0:1000]

In [0]:
data_df.shape

(1000, 5)

In [0]:
data_df['volume'] = data_df['volume'] / 1000000

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [0]:
data_df.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2.1636
1,125.239998,119.980003,119.940002,125.540001,2.3864
2,116.379997,114.949997,114.930000,119.739998,2.4895
3,115.480003,116.620003,113.500000,117.440002,2.0063
4,117.010002,114.970001,114.089996,117.330002,1.4086


### Divide the data into train and test sets

In [0]:
y = data_df['volume']
x = data_df.drop(['volume'], axis = 1)

In [0]:
x.shape, y.shape

((1000, 4), (1000,))

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
train_x,test_x,train_y,test_y = train_test_split(x,y,test_size = 0.3, random_state =5)

#### Convert Training and Test Data to numpy float32 arrays


In [0]:
import numpy as np
train_x =np.array(train_x).astype('float32')
test_x = np.array(test_x).astype('float32')
train_y =np.array(train_y).astype('float32')
test_y = np.array(test_y).astype('float32')

In [0]:
train_y=train_y.reshape(-1,1)

In [0]:
train_y.shape

(700, 1)

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn.preprocessing import Normalizer
transformer = Normalizer()
train_x = transformer.fit_transform(train_x)
test_x = transformer.transform(test_x)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
#We are initializing weights and Bias of (Input-Hidden Layer)
w1 = tf.random_normal(shape=(4,2))
b1 = tf.zeros(shape=(2)) 


In [0]:
#We are initializing weights and Bias of (Hidden-Output)
w2 = tf.random_normal(shape=(2,1))
b2 = tf.zeros(shape=(1))

2.Define a function to calculate prediction

In [0]:
def prediction(x, w1, b1,w2,b2):
    
    xw_matmul = tf.matmul(x, w1)
    net1 = tf.add(xw_matmul, b1)
    y=tf.sigmoid(net1)
    net2=tf.matmul(y, w2)+b2
    out=tf.sigmoid(net2)
    return net2,out

3.Loss (Cost) Function [Mean square error]

In [0]:
def loss(predicted_y, desired_y):
  return tf.reduce_mean(tf.square(predicted_y - desired_y))

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
def train(train_x, train_y, w1, b1,w2,b2, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w1,b1,w2,b2])
        
        net2,current_prediction = prediction(train_x, w1, b1,w2,b2)
        #current_loss =tf.nn.sigmoid_cross_entropy_with_logits(labels = train_y,logits =net2 )
        current_loss = loss(net2, train_y)
        print(current_loss)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw1,db1,dw2,db2 = t.gradient(current_loss,[w1, b1,w2,b2])
    
    #Update Weights at output layer
    w2 = w2 - learning_rate*dw2
    b2 = b2 - learning_rate*db2
    
     #Update Weights at hidden layer
    w1 = w1 - learning_rate*dw1
    b1 = b1 - learning_rate*db1

    #print("Loss at step {:d}: {:.3f}".format(i, currentloss)
    
    return w1, b1,w2,b2


## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [0]:
for i in range(100):
    
    w1,b1,w2,b2 = train(train_x, train_y, w1, b1,w2,b2)

tf.Tensor(275.86237, shape=(), dtype=float32)
tf.Tensor(272.20853, shape=(), dtype=float32)
tf.Tensor(268.86517, shape=(), dtype=float32)
tf.Tensor(265.8082, shape=(), dtype=float32)
tf.Tensor(263.01434, shape=(), dtype=float32)
tf.Tensor(260.46173, shape=(), dtype=float32)
tf.Tensor(258.12927, shape=(), dtype=float32)
tf.Tensor(255.9972, shape=(), dtype=float32)
tf.Tensor(254.04683, shape=(), dtype=float32)
tf.Tensor(252.2612, shape=(), dtype=float32)
tf.Tensor(250.62433, shape=(), dtype=float32)
tf.Tensor(249.1221, shape=(), dtype=float32)
tf.Tensor(247.7416, shape=(), dtype=float32)
tf.Tensor(246.47122, shape=(), dtype=float32)
tf.Tensor(245.30083, shape=(), dtype=float32)
tf.Tensor(244.22127, shape=(), dtype=float32)
tf.Tensor(243.22458, shape=(), dtype=float32)
tf.Tensor(242.30368, shape=(), dtype=float32)
tf.Tensor(241.45216, shape=(), dtype=float32)
tf.Tensor(240.66449, shape=(), dtype=float32)
tf.Tensor(239.93562, shape=(), dtype=float32)
tf.Tensor(239.26114, shape=(), dtype=fl

In [0]:
for i in range(0,100,5):
    print(i)
    w1,b1,w2,b2 = train(train_x, train_y, w1, b1,w2,b2)

0
tf.Tensor(231.23125, shape=(), dtype=float32)
5
tf.Tensor(231.23048, shape=(), dtype=float32)
10
tf.Tensor(231.22974, shape=(), dtype=float32)
15
tf.Tensor(231.22908, shape=(), dtype=float32)
20
tf.Tensor(231.2285, shape=(), dtype=float32)
25
tf.Tensor(231.22792, shape=(), dtype=float32)
30
tf.Tensor(231.22737, shape=(), dtype=float32)
35
tf.Tensor(231.22685, shape=(), dtype=float32)
40
tf.Tensor(231.22646, shape=(), dtype=float32)
45
tf.Tensor(231.22609, shape=(), dtype=float32)
50
tf.Tensor(231.22566, shape=(), dtype=float32)
55
tf.Tensor(231.22531, shape=(), dtype=float32)
60
tf.Tensor(231.22505, shape=(), dtype=float32)
65
tf.Tensor(231.22478, shape=(), dtype=float32)
70
tf.Tensor(231.22446, shape=(), dtype=float32)
75
tf.Tensor(231.22421, shape=(), dtype=float32)
80
tf.Tensor(231.22406, shape=(), dtype=float32)
85
tf.Tensor(231.22382, shape=(), dtype=float32)
90
tf.Tensor(231.22363, shape=(), dtype=float32)
95
tf.Tensor(231.22346, shape=(), dtype=float32)


### Get the shapes and values of W and b

In [0]:
w1

<tf.Tensor: id=6495, shape=(4, 2), dtype=float32, numpy=
array([[-0.1709895 ,  0.2733961 ],
       [-0.23268269, -0.8029887 ],
       [ 0.6300881 ,  0.58352613],
       [ 0.10248905,  0.3800675 ]], dtype=float32)>

In [0]:
w1.shape,w2.shape

(TensorShape([Dimension(4), Dimension(2)]),
 TensorShape([Dimension(2), Dimension(1)]))

In [0]:
w2

<tf.Tensor: id=6489, shape=(2, 1), dtype=float32, numpy=
array([[-0.7250861],
       [ 2.5486283]], dtype=float32)>

In [0]:
b1

<tf.Tensor: id=6498, shape=(2,), dtype=float32, numpy=array([-1.04344  ,  1.1389066], dtype=float32)>

In [0]:
b2

<tf.Tensor: id=6492, shape=(1,), dtype=float32, numpy=array([3.2699926], dtype=float32)>

### Model Prediction on 1st Examples in Test Dataset

In [0]:
y_pred,current_prediction = prediction(test_x, w1, b1,w2,b2)

In [0]:
y_pred

<tf.Tensor: id=6504, shape=(300, 1), dtype=float32, numpy=
array([[5.082682 ],
       [5.0801926],
       [5.0846634],
       [5.082935 ],
       [5.084079 ],
       [5.083778 ],
       [5.0810304],
       [5.0864286],
       [5.084632 ],
       [5.08372  ],
       [5.083723 ],
       [5.0833707],
       [5.0837154],
       [5.0812426],
       [5.0825434],
       [5.0849767],
       [5.081608 ],
       [5.083746 ],
       [5.077213 ],
       [5.083739 ],
       [5.0824966],
       [5.0839996],
       [5.08287  ],
       [5.0809817],
       [5.0863667],
       [5.084134 ],
       [5.0819645],
       [5.0811243],
       [5.083022 ],
       [5.082983 ],
       [5.081222 ],
       [5.0827637],
       [5.0836515],
       [5.084348 ],
       [5.0825095],
       [5.083247 ],
       [5.082354 ],
       [5.08355  ],
       [5.081661 ],
       [5.0814247],
       [5.082663 ],
       [5.082006 ],
       [5.083158 ],
       [5.084319 ],
       [5.082743 ],
       [5.0836077],
       [5.087432 ],
 

In [0]:
from sklearn.metrics import r2_score
print(r2_score(test_y,y_pred))

-0.002912658501541543


## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [0]:
iris = pd.read_csv("/content/drive/My Drive/greatlearning/11_Iris.csv")

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [0]:
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [0]:
y = pd.get_dummies(iris['Species'])

### Splitting the data into feature set and target set

In [50]:
x = iris.iloc[:,1:5]
x.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [52]:
y.head()

,Iris-setosa,Iris-versicolor,Iris-virginica
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [54]:
x.shape

(150, 4)

In [57]:
import numpy
from pandas import read_csv
from keras.models import Sequential
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Flatten
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [0]:
tf.disable_eager_execution()

In [61]:
model = Sequential()
model.add(Dense(4, input_dim=4, kernel_initializer='normal', activation='softmax'))
model.add(Dense(3, kernel_initializer='normal'))
# Compile model
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
   

W1201 11:58:37.534543 139865630336896 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1201 11:58:37.551270 139865630336896 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [0]:
train_x,test_x,train_y,test_y = train_test_split(x,y,test_size = 0.3, random_state =10)

### Model Training 

In [64]:
model.fit(train_x, train_y, validation_data=(test_x, test_y), epochs=50)

Train on 105 samples, validate on 45 samples
Epoch 1/50
105/105 [==============================] - 0s 142us/step - loss: 5.5262 - acc: 0.3429 - val_loss: 5.0145 - val_acc: 0.3111
Epoch 2/50
105/105 [==============================] - 0s 84us/step - loss: 5.5262 - acc: 0.3429 - val_loss: 5.0145 - val_acc: 0.3111
Epoch 3/50
105/105 [==============================] - 0s 80us/step - loss: 5.5262 - acc: 0.3429 - val_loss: 5.0145 - val_acc: 0.3111
Epoch 4/50
105/105 [==============================] - 0s 99us/step - loss: 5.5262 - acc: 0.3429 - val_loss: 5.0145 - val_acc: 0.3111
Epoch 5/50
105/105 [==============================] - 0s 83us/step - loss: 5.5262 - acc: 0.3429 - val_loss: 5.0145 - val_acc: 0.3111
Epoch 6/50
105/105 [==============================] - 0s 86us/step - loss: 5.5262 - acc: 0.3429 - val_loss: 5.0145 - val_acc: 0.3111
Epoch 7/50
105/105 [==============================] - 0s 82us/step - loss: 5.5262 - acc: 0.3429 - val_loss: 5.0145 - val_acc: 0.3111
Epoch 8/50
105/105 [===

### Model Prediction

In [65]:
model.predict(test_x)

array([[-0.38083145, -0.30514643,  0.38492385],
       [-0.3812707 , -0.30507883,  0.38713175],
       [-0.38085517, -0.30631244,  0.38106626],
       [-0.3810646 , -0.30523726,  0.385982  ],
       [-0.37985438, -0.3058172 ,  0.37978697],
       [-0.378627  , -0.30426922,  0.38146442],
       [-0.38092613, -0.30503264,  0.38599983],
       [-0.3803351 , -0.30501705,  0.3843182 ],
       [-0.38008755, -0.30590445,  0.38021594],
       [-0.38027808, -0.3048262 ,  0.3851001 ],
       [-0.38036662, -0.30486202,  0.3853848 ],
       [-0.3804306 , -0.30465043,  0.3862856 ],
       [-0.37968925, -0.3046404 ,  0.383462  ],
       [-0.37942183, -0.3055847 ,  0.37919894],
       [-0.38047683, -0.30611083,  0.38085517],
       [-0.38210464, -0.3055122 ,  0.38822258],
       [-0.38133672, -0.30541924,  0.38637727],
       [-0.3785736 , -0.30520833,  0.37826395],
       [-0.38043928, -0.30607378,  0.3807871 ],
       [-0.37981692, -0.30586445,  0.37936887],
       [-0.38241002, -0.3055889 ,  0.388

### Save the Model

In [0]:
model.save('iris_model.html')

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?